In [1]:
import math
from typing import Tuple
import time
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import dataset, DataLoader

In [12]:


transformer_model = nn.Transformer(d_model=1, nhead=1, num_encoder_layers=12, batch_first=True)
src = torch.rand((5, 1000, 1))
tgt = torch.rand((5, 2, 1))
out = transformer_model(src, tgt)

In [13]:
out.shape

torch.Size([5, 2, 1])